## Quicksort tornillos y tuercas

Nos dan una fila de $n$ tuercas y otra de los $n$ tornillos que les corresponden, pero en cualquier orden. Visualmente no podemos comparar dos tuercas o dos tornillos, pero sí podemos probar una tuerca con un tornillo y decidir cuál es mayor que la otra, o si corresponden.

A continuación, se presenta una implementación del algoritmo de quicksort para ordenar las tuercas y los tornillos.

1. Primero, se define una clase para representar las tuercas y los tornillos. Se implementan los métodos `__lt__`, `__gt__` y `__eq__` para comparar las tuercas y los tornillos con el operador `<`, `>` y `==` respectivamente: Se puede comparar una tuerca con un tornillo, pero no una tuerca con otra tuerca o un tornillo con otro tornillo.

In [1]:
import random
from dataclasses import dataclass
from pprint import pprint


@dataclass(eq=False)
class Pieza:
    size: int

    def tipo_opuesto(self):
        raise NotImplementedError("Subclase debe implementar tipo_opuesto")

    def _validate_opposite_type(self, other):
        if not isinstance(other, self.tipo_opuesto()):
            raise ValueError(
                f"No se puede comparar {self.__class__.__name__} "
                f"con algo que no sea {self.tipo_opuesto().__name__}"
            )

    def __lt__(self, other) -> bool:
        self._validate_opposite_type(other)
        return self.size < other.size

    def __gt__(self, other) -> bool:
        self._validate_opposite_type(other)
        return self.size > other.size

    def __eq__(self, other) -> bool:
        self._validate_opposite_type(other)
        return self.size == other.size


class Tornillo(Pieza):
    def tipo_opuesto(self):
        return Tuerca


class Tuerca(Pieza):
    def tipo_opuesto(self):
        return Tornillo

A continuación podemos ver algunas comparaciones entre tuercas y tornillos:

In [2]:
def print_lt_comp(a: Pieza, b: Pieza):
    try:
        print(f"{a} > {b} <=> {a > b}")
    except ValueError as e:
        print(f"{a} > {b} <=> {e}")
        
def print_eq_comp(a: Pieza, b: Pieza):
    try:
        print(f"{a} == {b} <=> {a == b}")
    except ValueError as e:
        print(f"{a} == {b} <=> {e}")

print_lt_comp(Tuerca(1), Tornillo(2))
print_lt_comp(Tornillo(3), Tuerca(2))
print_lt_comp(Tuerca(1), Tuerca(2))
print_lt_comp(Tornillo(1), Tornillo(2))
print()
print_eq_comp(Tuerca(3), Tornillo(3))
print_eq_comp(Tuerca(3), Tornillo(2))
print_eq_comp(Tuerca(1), Tuerca(1))
print_eq_comp(Tornillo(1), Tornillo(2))

Tuerca(size=1) > Tornillo(size=2) <=> False
Tornillo(size=3) > Tuerca(size=2) <=> True
Tuerca(size=1) > Tuerca(size=2) <=> No se puede comparar Tuerca con algo que no sea Tornillo
Tornillo(size=1) > Tornillo(size=2) <=> No se puede comparar Tornillo con algo que no sea Tuerca

Tuerca(size=3) == Tornillo(size=3) <=> True
Tuerca(size=3) == Tornillo(size=2) <=> False
Tuerca(size=1) == Tuerca(size=1) <=> No se puede comparar Tuerca con algo que no sea Tornillo
Tornillo(size=1) == Tornillo(size=2) <=> No se puede comparar Tornillo con algo que no sea Tuerca


2. Luego, se implementa el algoritmo de quicksort para ordenar las tuercas y los tornillos.

In [3]:
def quicksort_tornillos_y_tuercas(
    arr_tornillos: list[Tornillo],
    arr_tuercas: list[Tuerca],
) -> tuple[list[Tornillo], list[Tuerca]]:

    if not arr_tornillos or not arr_tuercas:
        return [], []

    # Elegir un tornillo como pivote
    pivot_tornillo = arr_tornillos[0]

    # Encontrar la tuerca correspondiente al tornillo pivote
    pivot_tuerca_idx = None
    for i, tuerca in enumerate(arr_tuercas):
        if tuerca == pivot_tornillo:
            pivot_tuerca_idx = i
            pivot_tuerca = tuerca
            break

    # Particionar tuercas usando el tornillo pivote
    tuercas_menores = []
    tuercas_mayores = []

    for i, tuerca in enumerate(arr_tuercas):
        if i == pivot_tuerca_idx:
            continue
        if tuerca < pivot_tornillo:
            tuercas_menores.append(tuerca)
        else:
            tuercas_mayores.append(tuerca)

    # Particionar tornillos usando la tuerca pivote
    tornillos_menores = []
    tornillos_mayores = []

    for i, tornillo in enumerate(arr_tornillos):
        if i == 0:  # Skip pivot
            continue
        if tornillo < pivot_tuerca:
            tornillos_menores.append(tornillo)
        else:
            tornillos_mayores.append(tornillo)

    # Recursivamente ordenar las sublistas
    tornillos_menores_ord, tuercas_menores_ord = quicksort_tornillos_y_tuercas(
        tornillos_menores, tuercas_menores
    )
    tornillos_mayores_ord, tuercas_mayores_ord = quicksort_tornillos_y_tuercas(
        tornillos_mayores, tuercas_mayores
    )

    # Combinar los resultados
    tornillos_ordenados = (
        tornillos_menores_ord + [pivot_tornillo] + tornillos_mayores_ord
    )
    tuercas_ordenadas = tuercas_menores_ord + [pivot_tuerca] + tuercas_mayores_ord

    return tornillos_ordenados, tuercas_ordenadas

3. Se prueba el algoritmo con un ejemplo:


In [4]:
def generate_example(size: int = 10) -> tuple[list[Tornillo], list[Tuerca]]:
    # Crear tornillos con tamaños de 1 a "size"
    sizes = list(range(1, size + 1))
    tornillos = [Tornillo(size) for size in sizes]
    random.shuffle(tornillos)

    # Crear las mismas tuercas pero en orden aleatorio
    tuercas = [Tuerca(size) for size in sizes]
    random.shuffle(tuercas)

    return tornillos, tuercas

tornillos, tuercas = generate_example(size=10)
print("Tornillos:\n")
pprint(tornillos)
print("\n")
print("Tuercas:\n")
pprint(tuercas)

Tornillos:

[Tornillo(size=2),
 Tornillo(size=10),
 Tornillo(size=6),
 Tornillo(size=3),
 Tornillo(size=5),
 Tornillo(size=9),
 Tornillo(size=4),
 Tornillo(size=7),
 Tornillo(size=1),
 Tornillo(size=8)]


Tuercas:

[Tuerca(size=10),
 Tuerca(size=3),
 Tuerca(size=7),
 Tuerca(size=9),
 Tuerca(size=2),
 Tuerca(size=6),
 Tuerca(size=4),
 Tuerca(size=5),
 Tuerca(size=1),
 Tuerca(size=8)]


4. Se ejecuta el algoritmo y se imprime el resultado:

In [5]:
tornillos_ord, tuercas_ord = quicksort_tornillos_y_tuercas(tornillos, tuercas)
pprint([(t, n) for t, n in zip(tornillos_ord, tuercas_ord)])

[(Tornillo(size=1), Tuerca(size=1)),
 (Tornillo(size=2), Tuerca(size=2)),
 (Tornillo(size=3), Tuerca(size=3)),
 (Tornillo(size=4), Tuerca(size=4)),
 (Tornillo(size=5), Tuerca(size=5)),
 (Tornillo(size=6), Tuerca(size=6)),
 (Tornillo(size=7), Tuerca(size=7)),
 (Tornillo(size=8), Tuerca(size=8)),
 (Tornillo(size=9), Tuerca(size=9)),
 (Tornillo(size=10), Tuerca(size=10))]


Como se puede ver, se resuelve el problema de ordenar las tuercas y los tornillos!